In [1]:
from __future__ import annotations

import asyncio
import nest_asyncio

# make asyncio.run() works in notebook
nest_asyncio.apply()

from metagpt.config2 import Config

cfg = Config.default()
cfg.llm

2024-03-17 18:01:30.536 | INFO     | metagpt.const:get_metagpt_package_root:29 - Package root set to /root/workspace/StreamChatPlayground/notebooks


LLMConfig(extra_fields=None, api_key='2089197edd83eddf547f600665da02e9.ohYNy9csNMqduxID', api_type=<LLMType.ZHIPUAI: 'zhipuai'>, base_url='https://api.openai.com/v1', api_version=None, model='glm-4', pricing_plan=None, access_key=None, secret_key=None, endpoint=None, app_id=None, api_secret=None, domain=None, max_token=4096, temperature=0.0, top_p=1.0, top_k=0, repetition_penalty=1.0, stop=None, presence_penalty=0.0, frequency_penalty=0.0, best_of=None, n=None, stream=False, logprobs=None, top_logprobs=None, timeout=60, proxy=None, calc_usage=True)

In [2]:
import sys
from metagpt.logs import logger
from metagpt.provider.zhipuai_api import ZhiPuAILLM

logger.remove()
logger.add(sys.stderr, level="DEBUG")
llm = ZhiPuAILLM(cfg.llm)

asyncio.run(llm.aask("你好"))

2024-03-17 18:02:59.269 | DEBUG    | metagpt.provider.base_llm:aask:126 - [{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': '你好'}]


Of course, I'm here to help. How can I assist you today?


"Of course, I'm here to help. How can I assist you today?"

## 任务拆解

In [40]:
import os
from langchain.prompts import PromptTemplate

def load_plaintext(filename):
    with open(filename, 'r', encoding="UTF-8") as f:
        return f.read()

path = "prompts/planning"

task_types = """**web scraping**: For scraping data from web pages."""

output_format = load_plaintext(os.path.join(path, "output.md"))

prompt = PromptTemplate(
    input_variables=[],
    template=load_plaintext(os.path.join(path, "template.yaml")),
)
prompt = prompt.partial(
    task_types=task_types,
    max_tasks=20,
    output=output_format,
)
# print(prompt.format(context="这里是指令"))

In [41]:
goal = """抓取 https://pitchhub.36kr.com/financing-flash 中'快讯'的内容，并整理成markdown存档。

大概的流程：
- 使用工具抓取网页中的可见文本
- 提取网页文本中'快讯'相关的内容。注意网页中可能包含导航，只需要抽取'快讯'的具体内容
- 对抽取结果进行归类，并保存成markdown表格: 快讯.md"""

planning_prompt = prompt.format(goal=goal)
# print(planning_prompt + "\n\n")

rsp = asyncio.run(llm.aask(msg=planning_prompt))

2024-03-17 20:00:17.622 | DEBUG    | metagpt.provider.base_llm:aask:126 - [{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'You are a helpful AI assistant.\n\n# User goal\n抓取 https://pitchhub.36kr.com/financing-flash 中\'快讯\'的内容，并整理成markdown存档。\n\n大概的流程：\n- 使用工具抓取网页中的可见文本\n- 提取网页文本中\'快讯\'相关的内容。注意网页中可能包含导航，只需要抽取\'快讯\'的具体内容\n- 对抽取结果进行归类，并保存成markdown表格: 快讯.md\n\n# Available Task Types:\n**web scraping**: For scraping data from web pages.\n\n# Task:\nBased on the user goal, write a plan or modify an existing plan of what you should do to achieve the goal. A plan consists of one to 20 tasks.\nIf you are modifying an existing plan, carefully follow the instruction, don\'t make unnecessary changes. Give the whole plan unless instructed to modify only one task of the plan.\nIf you encounter errors on the current task, revise and output the current single task only.\nOutput a list of jsons following the format:\n```json\n[\n    {{\n        "task_id": st

[
    {
        "task_id": "1",
        "dependent_task_ids": [],
        "instruction": "Use a web scraping tool to fetch the visible text from the given URL.",
        "task_type": "web scraping"
    },
    {
        "task_id": "2",
        "dependent_task_ids": ["1"],
        "instruction": "Extract the content related to '快讯' from the fetched text, excluding navigation elements.",
        "task_type": "web scraping"
    },
    {
        "task_id": "3",
        "dependent_task_ids": ["2"],
        "instruction": "Categorize the extracted content and format it into a markdown table.",
        "task_type": "data processing"
    },
    {
        "task_id": "4",
        "dependent_task_ids": ["3"],
        "instruction": "Save the formatted markdown table into a file named '快讯.md'.",
        "task_type": "file operation"
    }
]


In [42]:
import json
from metagpt.utils.common import OutputParser

try:
    tasks = json.loads(rsp)
except:
    code_block = OutputParser.parse_code(rsp, "json")
    tasks = json.loads(code_block)

tasks

[{'task_id': '1',
  'dependent_task_ids': [],
  'instruction': 'Use a web scraping tool to fetch the visible text from the given URL.',
  'task_type': 'web scraping'},
 {'task_id': '2',
  'dependent_task_ids': ['1'],
  'instruction': "Extract the content related to '快讯' from the fetched text, excluding navigation elements.",
  'task_type': 'web scraping'},
 {'task_id': '3',
  'dependent_task_ids': ['2'],
  'instruction': 'Categorize the extracted content and format it into a markdown table.',
  'task_type': 'data processing'},
 {'task_id': '4',
  'dependent_task_ids': ['3'],
  'instruction': "Save the formatted markdown table into a file named '快讯.md'.",
  'task_type': 'file operation'}]

## 任务执行

In [52]:
path = "prompts/task_plan"

task_types = """**web scraping**: For scraping data from web pages."""

output_format = load_plaintext(os.path.join(path, "output.md"))

prompt = PromptTemplate(
    input_variables=[],
    template=load_plaintext(os.path.join(path, "template.yaml")),
)
prompt = prompt.partial(
    plan_status="",
    current_task=tasks[0]['instruction'],
    task_guidance="",
    tools="""{'scrape_web_playwright': {'type': 'async_function', 'description': 'Asynchronously Scrape and save the HTML structure and inner text content of a web page using Playwright. ', 'signature': '(url)', 'parameters': "Args: url (str): The main URL to fetch inner text from. Returns: dict: The inner text content and html structure of the web page, keys are 'inner_text', 'html'.", 'tool_path': 'metagpt/tools/libs/web_scraping.py'}}""",
    output=output_format,
)
# print(prompt.format(goal="这里是指令"))

As an AI Engineer, you need to help user to achieve their goal step by step in a continuous Jupyter notebook. Since it is a notebook environment, don't use asyncio.run. Instead, use await if you need to call an async function.

# User goal
这里是指令



## Current Task
Use a web scraping tool to fetch the visible text from the given URL.

## Task Guidance
Write complete code for 'Current Task'. And avoid duplicating code from 'Finished Tasks', such as repeated import of packages, reading data, etc.
Specifically, 

# Tool Info

## Capabilities
- You can utilize pre-defined tools in any code lines from 'Available Tools' in the form of Python class or function.
- You can freely combine the use of any other public packages, like sklearn, numpy, pandas, etc..

## Available Tools:
Each tool is described in JSON format. When you call a tool, import the tool from its path first.
{'scrape_web_playwright': {'type': 'async_function', 'description': 'Asynchronously Scrape and save the HTML structure an

In [53]:
task_prompt = prompt.format(goal=goal)
print(task_prompt + "\n\n")

rsp = asyncio.run(llm.aask(msg=task_prompt))

2024-03-17 20:26:32.943 | DEBUG    | metagpt.provider.base_llm:aask:126 - [{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'As an AI Engineer, you need to help user to achieve their goal step by step in a continuous Jupyter notebook. Since it is a notebook environment, don\'t use asyncio.run. Instead, use await if you need to call an async function.\n\n# User goal\n抓取 https://pitchhub.36kr.com/financing-flash 中\'快讯\'的内容，并整理成markdown存档。\n\n大概的流程：\n- 使用工具抓取网页中的可见文本\n- 提取网页文本中\'快讯\'相关的内容。注意网页中可能包含导航，只需要抽取\'快讯\'的具体内容\n- 对抽取结果进行归类，并保存成markdown表格: 快讯.md\n\n\n\n## Current Task\nUse a web scraping tool to fetch the visible text from the given URL.\n\n## Task Guidance\nWrite complete code for \'Current Task\'. And avoid duplicating code from \'Finished Tasks\', such as repeated import of packages, reading data, etc.\nSpecifically, \n\n# Tool Info\n\n## Capabilities\n- You can utilize pre-defined tools in any code lines from \'Available Tools\' in the f

As an AI Engineer, you need to help user to achieve their goal step by step in a continuous Jupyter notebook. Since it is a notebook environment, don't use asyncio.run. Instead, use await if you need to call an async function.

# User goal
抓取 https://pitchhub.36kr.com/financing-flash 中'快讯'的内容，并整理成markdown存档。

大概的流程：
- 使用工具抓取网页中的可见文本
- 提取网页文本中'快讯'相关的内容。注意网页中可能包含导航，只需要抽取'快讯'的具体内容
- 对抽取结果进行归类，并保存成markdown表格: 快讯.md



## Current Task
Use a web scraping tool to fetch the visible text from the given URL.

## Task Guidance
Write complete code for 'Current Task'. And avoid duplicating code from 'Finished Tasks', such as repeated import of packages, reading data, etc.
Specifically, 

# Tool Info

## Capabilities
- You can utilize pre-defined tools in any code lines from 'Available Tools' in the form of Python class or function.
- You can freely combine the use of any other public packages, like sklearn, numpy, pandas, etc..

## Available Tools:
Each tool is described in JSON format. When you ca

In [55]:
code = OutputParser.parse_code(rsp, "python")
print(code)

# Import the tool for web scraping
from metagpt.tools.libs.web_scraping import scrape_web_playwright

# Define the URL to be scraped
url = 'https://pitchhub.36kr.com/financing-flash'

# Use the tool to scrape the web page for its inner text
async def fetch_pageInnerText(url):
    return await scrape_web_playwright(url)

# Execute the async function in a Jupyter notebook
import nest_asyncio
nest_asyncio.apply()

# This will store the result of the scraping
result = await fetch_pageInnerText(url)

# Extract the inner text from the result
pageInnerText = result['inner_text']

# Now, further processing will be needed to filter out the '快讯' content.
# However, since the current task is only to fetch the inner text, 
# we will stop here and the next steps will be handled in the subsequent tasks.



In [58]:
from metagpt.actions.di.execute_nb_code import ExecuteNbCode

execute_code = ExecuteNbCode()

result, success = await execute_code.run(code)

   1 # Import the tool for web scraping                                                                            
   2 from metagpt.tools.libs.web_scraping import scrape_web_playwright                                             
   3                                                                                                               
   4 # Define the URL to be scraped                                                                                
   5 url = 'https://pitchhub.36kr.com/financing-flash'                                                             
   6                                                                                                               
   7 # Use the tool to scrape the web page for its inner text                                                      
   8 async def fetch_pageInnerText(url):                                                                           
   9     return await scrape_web_playwright(url)                                                                   
  10                                                                                                               
  11 # Execute the async function in a Jupyter notebook                                                            
  12 import nest_asyncio                                                                                           
  13 nest_asyncio.apply()                                                                                          
  14                                                                                                               
  15 # This will store the result of the scraping                                                                  
  16 result = await fetch_pageInnerText(url)                                                                       
  17                                                                                                               
  18 # Extract the inner text from the result                                                                      
  19 pageInnerText = result['inner_text']                                                                          
  20                                                                                                               
  21 # Now, further processing will be needed to filter out the '快讯' content.                                    
  22 # However, since the current task is only to fetch the inner text,                                            
  23 # we will stop here and the next steps will be handled in the subsequent tasks.                               
  24                                                                                                               

In [60]:
result, success = await execute_code.run("print(pageInnerText)")
print(result)

  1 print(pageInnerText)                                                                                           

首页
融资快报
融资事件
项目库
机构库
项目集
定向对接
融通创新
公司/项目名/投资机构/赛道
 
返回36氪
登录
融资快报
文章
Alithea Genomics获320万美元种子轮融资，提供RNA测序服务 | 海外New Things
该公司研发的产品适用于大型转录学研究，如生物标志物发现、抗体开发和药物筛查。
12小时前
海若镜
文章
Tubulis获1.388亿美元B轮融资，实体瘤ADC管线再得资金加持 | 海外New Things
Tubulis将新型专有技术与对特定疾病的生物洞察力相结合，生产出独特匹配的蛋白质药物偶联物。
12小时前
海若镜
文章
Asgard Therapeutics获3000万欧元融资，强生、诺和控股等支持其基因治疗 | 海外New Things
支持其主打项目AT-108在2026年前达到支持递交IND申请的状态。
12小时前
海若镜
文章
「锐正基因」专注非病毒载体体内基因编辑药物开发；先博生物完成2亿元A+轮融资
36氪医疗健康行业周报
锐正基因
Pre-A轮
江苏省
2021年成立
端到端体内基因编辑技术与药物开发与生产商
12小时前
胡安2022
文章
Zephyr AI完成1.11亿美元A轮融资，AI+精准医疗技术获礼来支持 | 海外 New Things
用AI加速临床试验的速度和成功率。
昨天
海若镜
文章
X-Therma获2240万美元B轮融资，研发细胞及器官低温保存技术 | 海外New Things
X-Therma通过融合生物保存平台开创突破性冷链技术，推动再生医学发展，为有需要的患者提供安全、按需的器官、工程组织、细胞和基因疗法、疫苗等。
2024-03-15
海若镜
快讯
“卓宇智能”获近亿元A轮融资
36氪获悉，“卓宇智能”近日完成近亿元A轮融资，由金雨茂物领投，常州拓扑投资跟投。本轮融资将用于渠道和交付能力的扩建。原文链接
2024-03-15
文章
专注军民融合智能光电产品，「卓宇智能」获近亿元A轮融资｜硬氪首发
从底层基础算法做起，逐步向产业纵深探索。
2024-03-15
黄 楠
文章
上海百奥恒完成亿元A+轮融资、黑石加码数据中心资产、深圳加大新型基础设施建设投入| PropTech周刊97期
聚焦改变地产行业的科技及创新力量
2024-03-15
宋虹姗
文章
成立机器人事业部，「本末